<a href="https://colab.research.google.com/github/SuryaTejaReddyS/Book-Recommendation-System-/blob/main/book_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  !pip install pandas==1.5.0

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
df = pd.read_csv('books.csv', error_bad_lines=False)
df.head(1)

<ipython-input-9-5bc9663cc121>:10: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('books.csv', error_bad_lines=False)
Skipping line 3350: expected 12 fields, saw 13
Skipping line 4704: expected 12 fields, saw 13
Skipping line 5879: expected 12 fields, saw 13
Skipping line 8981: expected 12 fields, saw 13



,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.


In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
df.isnull().sum()
df.describe()

,bookID,average_rating,isbn13,num_pages,ratings_count,text_reviews_count
count,11123.000000,11123.000000,1.112300e+04,11123.000000,1.112300e+04,11123.000000
mean,21310.856963,3.934075,9.759880e+12,336.405556,1.794285e+04,542.048099
std,13094.727252,0.350485,4.429758e+11,241.152626,1.124992e+05,2576.619589
min,1.000000,0.000000,8.987060e+09,0.000000,0.000000e+00,0.000000
25%,10277.500000,3.770000,9.780345e+12,192.000000,1.040000e+02,9.000000
50%,20287.000000,3.960000,9.780582e+12,299.000000,7.450000e+02,47.000000
75%,32104.500000,4.140000,9.780872e+12,416.000000,5.000500e+03,238.000000
max,45641.000000,5.000000,9.790008e+12,6576.000000,4.597666e+06,94265.000000


top 10 averaging rating book

In [ ]:
top_ten = df[df['ratings_count'] > 1000000]
top_ten.sort_values(by='average_rating', ascending=False)
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(10, 10))
data = top_ten.sort_values(by='average_rating', ascending=False).head(10)
sns.barplot(x="average_rating", y="title", data=data, palette='inferno')

Authors and no.of books

In [ ]:
most_books = df.groupby('authors')['title'].count().reset_index().sort_values('title', ascending=False).head(10)

plt.figure(figsize=(10, 4))
ax = sns.barplot(x='title', y='authors', data=most_books, palette='inferno')
ax.set_title("Top 10 authors with most books")
ax.set_xlabel("Total number of books")

for p in ax.patches:
    ax.annotate(f'{p.get_width():.0f}',
                (p.get_width() + 0.2, p.get_y() + p.get_height() / 2),
                xytext=(5, 0),
                textcoords='offset points',
                va='center')

plt.show()

most rated books

In [ ]:
most_rated = df.set_index('title').sort_values('ratings_count', ascending=False).head(10)
plt.figure(figsize=(10, 4))
ax = sns.barplot(x='ratings_count', y=most_rated.index, data=most_rated, palette='inferno')
for p in ax.patches:
    ax.annotate(f'{p.get_width():.0f}',
                (p.get_width() + 0.2, p.get_y() + p.get_height() / 2),
                xytext=(1, 0),
                textcoords='offset points',
                va='top')

plt.show()

average rating of all books

In [ ]:
df.average_rating = df.average_rating.astype(float)
fig, ax = plt.subplots(figsize=[10,4])
sns.distplot(df['average_rating'],ax=ax)
ax.set_title('Average rating for all books',fontsize=20)
ax.set_xlabel('Average rating',fontsize=13)

In [ ]:
ax = sns.relplot(data=df, x="average_rating", y="ratings_count", color = 'black', sizes=(100, 200), height=7, marker='*')
plt.title("Relation between Rating counts and Average Ratings",fontsize = 15)
ax.set_axis_labels("Average Rating", "Ratings Count")

In [ ]:
plt.figure(figsize=(15,10))
ax = sns.relplot(x="average_rating", y="  num_pages", data = df, color = 'black',sizes=(100, 200), height=7, marker='*')
ax.set_axis_labels("Average Rating", "Number of Pages")

In [ ]:
df2 = df.copy()

In [ ]:
df2.loc[ (df2['average_rating'] >= 0) & (df2['average_rating'] <= 1), 'rating_between'] = "between 0 and 1"
df2.loc[ (df2['average_rating'] > 1) & (df2['average_rating'] <= 2), 'rating_between'] = "between 1 and 2"
df2.loc[ (df2['average_rating'] > 2) & (df2['average_rating'] <= 3), 'rating_between'] = "between 2 and 3"
df2.loc[ (df2['average_rating'] > 3) & (df2['average_rating'] <= 4), 'rating_between'] = "between 3 and 4"
df2.loc[ (df2['average_rating'] > 4) & (df2['average_rating'] <= 5), 'rating_between'] = "between 4 and 5"

In [ ]:
rating_df = pd.get_dummies(df2['rating_between'])
language_df = pd.get_dummies(df2['language_code'])

In [ ]:
features = pd.concat([rating_df,
                      language_df,
                      df2['average_rating'],
                      df2['ratings_count']], axis=1)

In [ ]:
X = features
y = df2['average_rating']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
features = min_max_scaler.fit_transform(features)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
predictions = rf_model.predict(X_test)

In [ ]:
model = neighbors.NearestNeighbors(n_neighbors=10, algorithm='ball_tree')
model.fit(features)
dist, idlist = model.kneighbors(features)

In [ ]:
predictions = rf_model.predict(X_test)

K-Nearest Neighbourhood

In [ ]:
def BookRecommender(book_name):
    book_list_name = []
    book_id = df2[df2['title'] == book_name].index
    book_id = book_id[0]
    for newid in idlist[book_id]:
        book_list_name.append(df2.loc[newid].title)
    return book_list_name
BookNames = BookRecommender('Notes from a Small Island')
BookNames



In [ ]:
model = neighbors.KNeighborsRegressor(n_neighbors=5)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
print("Mean Absolute Error:", mae)

Random Forest Regressor



In [ ]:
def BookRecommender(book_name):
    book_list_name = []
    book_id = df2[df2['title'] == book_name].index
    book_id = book_id[0]
    for newid in idlist[book_id]:
        book_list_name.append(df2.loc[newid].title)
    return book_list_name, rf_model.predict(features[book_id].reshape(1, -1))
book_list, predicted_rating = BookRecommender('Notes from a Small Island')
print("Recommended books:", book_list)
print("Predicted average rating for the queried book:", predicted_rating)



In [ ]:
mae = mean_absolute_error(y_test, predictions)
print("Mean Absolute Error:", mae)

Recommend popular books

In [ ]:
def recommend_popular_books(df, top_n=3):
    popular_books = df.sort_values(by='ratings_count', ascending=False).head(top_n)
    popular_book_titles = popular_books['title'].values.tolist()
    return popular_book_titles

top_popular_books = recommend_popular_books(df)
print("Top 3 popular books:", top_popular_books)
